In [2]:
import numpy as np
import pandas as pd
import math

import plotly.graph_objects as go

from Vec2d import Vec2d
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance



In [5]:
def create_bm_2d(start_pos=[0,0], n_steps=10000, speed=6):
    # Init velocity vector
    velocity = Vec2d(speed, 0)

    #Init DF
    BM_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{ 
        'x_pos': start_pos[0], 
        'y_pos': start_pos[1], 
    }])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    for i in range(n_steps - 1):
        #turn_angle = np.random.choice([0, np.pi/2, np.pi, 3*np.pi/2])
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)
        temp_df = pd.DataFrame([{
            'x_pos': BM_2d_df.x_pos[i] + velocity.x,
            'y_pos': BM_2d_df.y_pos[i] + velocity.y
        }])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
    
    return BM_2d_df

# Metrics

## Path Length

In [21]:
# Compute path-length with Euclidean distance


def create_pl_bm_2d(trajectory):

    dis_BM = np.array([ 
        distance.euclidean(
            trajectory.iloc[i-1], trajectory.iloc[i]) for i in range(1, trajectory.shape[0]
        )
    ])

    pl_dis_bm = np.cumsum(dis_BM)
    return pl_dis_bm

def plot_pl_bm_2d(pl_dis_bm, fig):
    fig.add_trace(go.Scatter(
        x = np.arange(len(pl_dis_bm)),
        y = pl_dis_bm,
        marker = dict( size = 2 ),
        line = dict( width = 2 ),
        name = "path_length",
        showlegend = True
    ))

In [22]:
speeds = [3, 6]
fig_path_length = go.Figure()

for speed in speeds:
    BM_2d_df_s = create_bm_2d(start_pos=[0,0], n_steps=1000, speed=speed)
    pl_bm_2d_s = create_pl_bm_2d(BM_2d_df_s)

    plot_pl_bm_2d(pl_dis_bm=pl_bm_2d_s, fig=fig_path_length)

fig_path_length.show()

In [29]:
# Trajectories from csv

fig_3d = go.Figure()

BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

pl_bm_2d_6 = create_pl_bm_2d(BM_2d_df_6)


# Crear Figura
fig_3d.add_trace(go.Scatter3d(
    x=BM_2d_df_6.x_pos, 
    y=BM_2d_df_6.y_pos, 
    z=BM_2d_df_6.index,
    marker=dict(
        size=3,
        colorscale='haline',
    ),
    line=dict(
        color='blue',
        width=2
    ),
    mode = "lines",
    name = "BW_2d",
    showlegend = True
))

fig_3d.add_trace(go.Scatter3d(
    x = [BM_2d_df_6.x_pos[0]],
    y = [BM_2d_df_6.y_pos[0]],
    z = [0],
    name="BM_2D_Start",
    mode="markers",
    showlegend = True
))

fig_3d.add_trace(go.Scatter3d(
    x = [BM_2d_df_6["x_pos"].iloc[-1]],
    y = [BM_2d_df_6["y_pos"].iloc[-1]],
    z = [1000],
    name="BM_2D_End",
    mode="markers",
    showlegend = True
))

# FIgure Params
fig_3d.update_layout(
    title_text = "BM 2d Trajectory in 3D",
    autosize = False,
    width = 800,
    height = 800,
    scene_camera = dict(
        up = dict( x=0, y=1, z=0 ),
        center = dict( x=0, y=0, z=0 ),
        eye = dict( x=0, y=0, z=1 ),
    ),
    scene = dict(
        xaxis = dict(title='x_pos (mm)'),
        yaxis = dict(title='y_pos (mm)'),
        zaxis = dict(title='time', nticks = 20),
    )
)

fig_3d.show()

## Mean Squared Displacement

In [ ]:
displacement_vec = np.array([ distance.euclidean(CRW_2d_df_9.iloc[i - n], CRW_2d_9.iloc[i])
        for i in range(n, CWR_2d_df_9.shape[0], 1)])

In [5]:
# Load DF with MSD

met_df_2 = pd.read_csv('../metrics/met_df_2.csv')

fig_msd = go.Figure()

fig_msd.add_trace(go.Scatter(
    x = met_df_2.index,
    y = met_df_2.MSD_BM,
    mode = 'lines',
    name = 'MSD BM 6',
    showlegend = True
))

fig_msd.add_trace(go.Scatter(
    x = met_df_2.index,
    y = met_df_2.MSD_CRW,
    mode = 'lines',
    name = 'MSD CRW 6, c=0.9',
    showlegend = True
))

fig_msd.show()